# Will it be an early Spring?

On February 2<sup>nd</sup> every year Punxsutawney Phil makes a prediction about if there will be an early Spring or if Winter will continue for 6 more weeks (till about mid-March). He is however not very accurate (well, according to [The Inner Circle](https://www.groundhog.org/inner-circle) he is 100% correct but the human handler may not interpret his response correctly). The overall goal is to be able to predict if it will be an early Spring.

For this project you must go through most steps in the checklist. You must write responses for all items however sometimes the item will simply be "does not apply". Some of the parts are a bit more nebulous and you simply show that you have done things in general (and the order doesn't really matter). Keep your progress and thoughts organized in this document and use formatting as appropriate (using markdown to add headers and sub-headers for each major part). Do not do the final part (launching the product) and your presentation will be done as information written in this document in a dedicated section, no slides or anything like that. It should however include the best summary plots/graphics/data points.

You are intentionally given very little information thus far. You must communicate with your client (me) for additional information as necessary. But also make sure that your communications are efficient, thought out, and not redundant as your client might get frustrated and "fire" you (this only applies to getting information from your client, this does not necessary apply to asking for help with the actual project itself).

Each group from 200-level and 300-level sections with the best results on the 10% of the data that I kept for myself will earn +5 extra credit (if multiple groups are close points may be given to multiple groups).

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
def load_weather_data():
    """
    Loads the CSV file from the web which contains our data.
    """
    return pd.read_csv('/Users/marchesem@moravian.edu/Downloads/weather.csv')

In [7]:
data = load_weather_data()

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2211 entries, 0 to 2210
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           2211 non-null   object 
 1   max_temp       2167 non-null   float64
 2   min_temp       2170 non-null   float64
 3   avg_temp       2160 non-null   float64
 4   precipitation  2208 non-null   float64
 5   snowfall       2198 non-null   float64
 6   snowdepth      2174 non-null   float64
dtypes: float64(6), object(1)
memory usage: 121.0+ KB


In [9]:
data.head()

,date,max_temp,min_temp,avg_temp,precipitation,snowfall,snowdepth
0,1947-01-01,30.0,24.0,27.0,0.12,0.5,1.0
1,1947-01-02,44.0,29.0,36.5,0.49,0.0,0.0
2,1947-01-03,54.0,41.0,47.5,0.04,0.0,NaN
3,1947-01-04,42.0,24.0,33.0,0.00,0.0,NaN
4,1947-01-05,33.0,25.0,29.0,0.00,0.0,NaN


In [10]:
data.describe()

,max_temp,min_temp,avg_temp,precipitation,snowfall,snowdepth
count,2167.000000,2170.000000,2160.000000,2208.000000,2198.000000,2174.000000
mean,36.191970,18.410138,27.284028,0.111475,0.463889,2.503059
std,12.252389,12.755078,11.704472,0.228599,1.196003,4.282987
min,-6.000000,-26.000000,-15.000000,0.000000,0.000000,0.000000
25%,28.000000,10.000000,19.375000,0.000000,0.000000,0.000000
50%,36.000000,20.000000,28.000000,0.010000,0.000000,1.000000
75%,44.000000,28.000000,35.125000,0.120000,0.500000,3.000000
max,75.000000,51.000000,60.000000,2.560000,21.000000,30.000000
